In [1]:
import pandas as pd
import cobra
import numpy as np
import matplotlib.pyplot as plt

from micom import Community

from copy import deepcopy

from scipy.optimize import linprog
import scipy

In [2]:
def create_stoichiometry_matrix(model):
    metabolites = model.metabolites 
    reactions = model.reactions 
    S = np.zeros((len(metabolites), len(reactions)))
    
    met_id = dict()
    rec_id = dict()
    for i,reaction in enumerate(model.reactions):
        rec_id[reaction.id] = i
        for metabolite, stoich in reaction.metabolites.items():
            met_id[metabolite.id] = int(metabolites.index(metabolite))
            S[metabolites.index(metabolite), i] = stoich
    return S, met_id, rec_id 

class Model():
    def __init__(self, model, biomass_function):
        """ This is a new class of metabolic model, capable of flux balance analysis
        Attributes:
        models (list): CobraPy models of single organisms which will be used in construction
        biomass_reactions (list): List of strings containing the ids for the growth reactions
        """
        self.biomass_function = biomass_function
        self.model = model
        self.id = model.id
        # Compute stoichimetry_matrix
        S, met_id, rec_id = create_stoichiometry_matrix(model)
        self.num_reactions = S.shape[1]
        self.num_metabolites = S.shape[0]
        self.stoichiometry_matrix = scipy.sparse.csr_matrix(S)
        self.met_id = met_id
        self.rec_id = rec_id 
        # Set objective
        idx = self.rec_id[biomass_function]
        c = np.zeros(self.num_reactions)
        c[idx] = 1
        self.objective_c = c
        # Set bounds
        self._reset_bounds()
    
    @property
    def reactions(self):
        return self.model.reactions
    @property
    def exchanges(self):
        return self.model.exchanges
    @property
    def metabolites(self):
        return self.model.metabolites
    @property
    def medium(self):
        return self.model.medium

    def set_medium(self, medium):
        ex_ids = [ex.id for ex in self.exchanges]
        new_med = {}
        for key,val in medium.items():
            if key in ex_ids:
                new_med[key] = val
        self.model.medium = new_med
        self._reset_bounds()
        
    def optimize(self, disp=False):
        sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
        sol["fun"] = -sol["fun"] # As we have to minimize
        return sol 
    
    def slim_optimize(self, disp=False):
        sol = self.optimize(disp=disp)
        return sol["fun"]

    def summary(self):
        sol = self.optimize()
        flux = sol["x"]
        ex_ids = [ex.id for ex in self.exchanges]
        fluxes = []
        for ex in ex_ids:
            idx = self.rec_id[ex]
            fluxes.append(flux[idx])
        summary_df = pd.DataFrame({"Exchange reaction": ex_ids, "Flux": fluxes})
        summary_df.sort_values(["Flux"], inplace=True)
        return summary_df

    def _reset_bounds(self):
        self.bounds = []
        for rec in self.model.reactions:
            self.bounds.append((rec.lower_bound, rec.upper_bound))

    def __add__(self, model2):
        """ Adding another model creates a community model """
        return CommunityModel([self,model2], [1.,1.])
    
class CommunityModel(Model):
    def __init__(self, models, weight):
        self.models = models
        self.id = "|".join([model.id for model in models])
        self.shared_exchanges = []
        self.weight = weight
        for model in models:
            for rec in model.exchanges:
                if rec.id not in self.shared_exchanges:
                    self.shared_exchanges.append(rec.id)
        # Create community stoichimetry matrix with shuttel reactions!
        self._shifts = [len(self.shared_exchanges)]
        for i,model in enumerate(models):
            self._shifts.append(self._shifts[i] + model.num_reactions)
        S_EX = -np.eye(self._shifts[0])
        matrices = [S_EX] + [m.stoichiometry_matrix.todense() for m in models]
        S = scipy.linalg.block_diag(*matrices)
        self.num_reactions = S.shape[1]
        self.num_metabolites = S.shape[0]
        for i, id in enumerate(self.shared_exchanges):
            for j,model in enumerate(models):
                if id in model.rec_id:
                    S[i,self._shifts[j] + model.rec_id[id]] = 1
        self.stoichiometry_matrix = scipy.sparse.csr_matrix(S)
        # Cretae objective:
        self._weighted_objective(weight)
        # Create bounds
        self._reset_bounds()
    
    @property
    def reactions(self):
        return [model.reactions for model in self.models]
    @property
    def exchanges(self):
        return self.shared_exchanges
    @property
    def metabolites(self):
        return [model.metabolites for model in self.models]
    @property
    def medium(self):
        medium = {}
        for model in self.models:
            for key,val in model.medium:
                medium[key] = val
        return medium

    def _weighted_objective(self, weight):
        self.weight = weight
        self.objective_c = np.zeros(self._shifts[0])
        for i,model in enumerate(self.models):
            self.objective_c = np.append(self.objective_c, weight[i]*model.objective_c)
    def _reset_bounds(self):
        self.bounds = []
        for id in self.shared_exchanges:
            min_lower_bound = 0
            for model in self.models:
                if id in model.rec_id:
                    rec = model.reactions.get_by_id(id)
                    if rec.lower_bound < min_lower_bound:
                        min_lower_bound = rec.lower_bound 
            self.bounds.append((min_lower_bound, 1000))
        for model in self.models:
            self.bounds += model.bounds

    def set_medium(self, medium):
        for model in self.models:
            model.set_medium(medium)
        self._reset_bounds()

    def summary(self):
        sol = self.optimize()
        flux = sol["x"]
        ex_ids = self.shared_exchanges
        ex_flux = flux[:len(ex_ids)]
        df_ex = pd.DataFrame({"Exchanges": ex_ids, "Flux": ex_flux})
        df_ex.sort_values(["Flux"], inplace=True)
        for i,model in enumerate(self.models):
            shuttel_ids = [ex.id for ex in model.exchanges]
            id = str(model.id) + " Shuttel Flux"
            df_ex[id] = 0.
            for sh in shuttel_ids:
                idx = model.rec_id[sh]
                df_ex[id][ex_ids.index(sh)] = flux[self._shifts[i] +idx]
        
        return df_ex
    
    def set_weights(self, weight):
        self._weighted_objective(weight)
    
    def get_model_growths(self):
        mask = self.objective_c != 0
        sol = self.optimize()
        flux = sol["x"]
        return flux[mask]

In [3]:
model_DP = cobra.io.read_sbml_model("models/consistent_DP_SNM.xml")
model_SA = cobra.io.read_sbml_model("models/consistent_iYS854_SNM.xml")
print("Growth: ", model_DP.slim_optimize())
print("Growth: ", model_SA.slim_optimize())

Growth:  0.282365392532252
Growth:  2.558694612613397


In [4]:
BIOMASS_DP = "Growth" 
BIOMASS_SA = "BIOMASS_iYS_wild_type"
models = [model_DP.copy(), model_SA.copy()]

In [5]:
def create_micom_model(weights):
    row1 = ["SA", "Staphylococcus", "Staphylococcus aureus", weights[0], len(model_SA.reactions),len(model_SA.metabolites), "models\\consistent_iYS854_SNM.xml" ]
    row2 = ["DP", "Dolosigranulum", "Dolosigranulum pigrum", weights[1],len(model_DP.reactions),len(model_DP.metabolites), "models\\consistent_DP_SNM.xml" ]
    taxonomy = pd.DataFrame(dict(zip(['id', 'genus', 'species', 'abundance', 'reactions', 'metabolites', 'file'],list(zip(row1,row2)))))
    com = Community(taxonomy)
    reactions_DP = [rec for rec in com.reactions if "__DP" in rec.id]
    for rec in reactions_DP:
        id = rec.id[:-4]
        org_rec = model_DP.reactions.get_by_id(id)
        rec.lower_bound = org_rec.lower_bound 
        rec.upper_bound = org_rec.upper_bound
    reactions_SA = [rec for rec in com.reactions if "__SA" in rec.id]
    for rec in reactions_SA:
        id = rec.id[:-4]
        org_rec = model_SA.reactions.get_by_id(id)
        rec.lower_bound = org_rec.lower_bound 
        rec.upper_bound = org_rec.upper_bound

    return com

In [6]:
com = create_micom_model([0.09,0.9])

Building ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

In [7]:
com.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
DP,0.909091,0.282365,1152,827
SA,0.090909,2.558695,928,729
medium,NaN,NaN,111,111


In [8]:
def adapt_micom_model(com):
    reactions_DP = [rec for rec in com.reactions if "__DP" in rec.id]
    reactions_SA = [rec for rec in com.reactions if "__SA" in rec.id]
    
    sh_DP = [ex for ex in reactions_DP if "EX_" in ex.id]
    sh_SA = [ex for ex in reactions_SA if "EX_" in ex.id]

    # remove scalling by abundance ...
    new_shuttles_DP = []
    for shuttle in sh_DP:
        new_shuttle = cobra.Reaction(id=shuttle.id, name=shuttle.name, lower_bound=shuttle.lower_bound, upper_bound=shuttle.upper_bound)
        new_shuttle.global_id = shuttle.global_id
        new_shuttle.community_id = shuttle.community_id
        new_mets = {}
        for key, val in shuttle.metabolites.items():
            if abs(val) != 1.:
                new_mets[key] = 1.
            else:
                new_mets[key] = val 
        new_shuttle.add_metabolites(new_mets)
        new_shuttles_DP.append(new_shuttle)

    # remove scalling by abundance ...
    new_shuttles_SA = []
    for shuttle in sh_SA:
        new_shuttle = cobra.Reaction(id=shuttle.id, name=shuttle.name, lower_bound=shuttle.lower_bound, upper_bound=shuttle.upper_bound)
        new_shuttle.global_id = shuttle.global_id
        new_shuttle.community_id = shuttle.community_id
        new_mets = {}
        for key, val in shuttle.metabolites.items():
            if abs(val) != 1.:
                new_mets[key] = 1.
            else:
                new_mets[key] = val 
        new_shuttle.add_metabolites(new_mets)
        new_shuttles_SA.append(new_shuttle)

    com.remove_reactions(sh_DP + sh_SA)
    com.add_reactions(new_shuttles_DP + new_shuttles_SA)
    return com

In [9]:
def create_community_summary(com):
    com.optimize()
    exchanges = []
    flux = []
    flux_DP = []
    flux_SA = []
    for ex in com.exchanges:
        f = ex.flux 

        exchanges.append(ex.id)
        flux.append(f)

        try:
            r = com.reactions.get_by_id(ex.id[:-1] + "e__DP")
            flux_DP.append(r.flux)
        except:
            flux_DP.append(0)

        try:
            r = com.reactions.get_by_id(ex.id[:-1] + "e__SA")
            flux_SA.append(r.flux)
        except:
            flux_SA.append(0)
    return pd.DataFrame({"Exchanges":exchanges, "Flux":flux, "iYS854 Shuttel Flux": flux_SA, "DP_83VPs_KB5 Shuttel Flux": flux_DP}).sort_values("Flux", ascending=True).round(decimals=6)

In [10]:
df_micom_unadjusted = create_community_summary(com)
df_micom_unadjusted.to_csv("summary_micom_unadjusted_weights_0_9__0_09.csv")

In [11]:
# Removes abundance scaling
com = adapt_micom_model(com)

In [12]:
com.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
DP,0.909091,0.282365,1152,827
SA,0.090909,1.471297,928,729
medium,NaN,NaN,111,111


In [13]:
model1 = Model(model_SA, biomass_function=BIOMASS_SA)
model2 = Model(model_DP, biomass_function=BIOMASS_DP)

model = model1 + model2
model.set_weights(com.abundances.to_list())

In [14]:
model.get_model_growths()

array([1.47129662, 0.28236539])

In [15]:
model.get_model_growths()

array([1.47129662, 0.28236539])

In [16]:
df_scipy = model.summary()

<ipython-input-2-c9a3cd5dedf6>:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ex[id][ex_ids.index(sh)] = flux[self._shifts[i] +idx]


In [17]:
df_micom = create_community_summary(com)

In [18]:
df_scipy.to_csv("summary_scipy_weights_0_9__0_09.csv")
df_micom.to_csv("summary_micom_adjusted_weights_0_9__0_09.csv")

In [19]:
df_scipy

,Exchanges,Flux,iYS854 Shuttel Flux,DP_83VPs_KB5 Shuttel Flux
42,EX_o2_e,-20.000000,-19.874221,-0.125779
56,EX_ser__L_e,-10.000000,-10.000000,0.000000
51,EX_glc__D_e,-10.000000,-0.000000,-10.000000
24,EX_cys__L_e,-10.000000,-0.265291,-9.734709
52,EX_pyr_e,-10.000000,-10.000000,0.000000
...,...,...,...,...
10,EX_h2o_e,13.604625,23.604625,-10.000000
39,EX_nh4_e,19.684068,19.684068,0.000000
19,EX_lac__L_e,21.124497,21.124497,0.000000
98,EX_ser__D_e,25.506917,0.000000,25.506917


In [20]:
df_micom

,Exchanges,Flux,iYS854 Shuttel Flux,DP_83VPs_KB5 Shuttel Flux
42,EX_o2_m,-20.000000,-19.874221,-0.125779
51,EX_glc__D_m,-10.000000,0.000000,-10.000000
11,EX_arg__L_m,-10.000000,-9.919116,-0.080884
10,EX_h2o_m,-10.000000,0.000000,-10.000000
86,EX_ala__L_m,-10.000000,0.000000,-10.000000
...,...,...,...,...
83,EX_12ppd__S_m,16.853028,0.000000,16.853028
32,EX_etoh_m,21.949303,21.949303,0.000000
5,EX_acald_m,32.641152,0.000000,32.641152
39,EX_nh4_m,65.861874,44.008937,21.852937


In [21]:
df_micom_unadjusted

,Exchanges,Flux,iYS854 Shuttel Flux,DP_83VPs_KB5 Shuttel Flux
42,EX_o2_m,-20.000000,-20.000000,-20.000000
11,EX_arg__L_m,-10.000000,-10.000000,-10.000000
51,EX_glc__D_m,-10.000000,-10.000000,-10.000000
86,EX_ala__L_m,-9.090909,0.000000,-10.000000
88,EX_gly_m,-9.090909,0.000000,-10.000000
...,...,...,...,...
18,EX_lac__D_m,2.806008,30.866093,0.000000
43,EX_orn_m,8.108288,-10.000000,9.919116
5,EX_acald_m,31.277197,0.000000,34.404917
39,EX_nh4_m,38.602927,28.846624,39.578557
